# Hardt

In [ ]:
from pathlib import Path

import joblib
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from aif360.datasets import StandardDataset
from aif360.algorithms.postprocessing.eq_odds_postprocessing import (
    EqOddsPostprocessing,
)
from helpers.fairness_measures import accuracy, equalised_odds_p
from helpers.finance import preprocess
from helpers.plot import group_box_plots

In [ ]:
from helpers import export_plot

## Load data



In [ ]:
artifacts_dir = Path("../../../artifacts")

In [ ]:
# override data_dir in source notebook
# this is stripped out for the hosted notebooks
artifacts_dir = Path("../../../../artifacts")

Location of the data

In [ ]:
data_dir = artifacts_dir / "data" / "adult"
preprocess(data_dir)

In [ ]:
train = pd.read_csv(data_dir / "processed" / "train-one-hot.csv")
val = pd.read_csv(data_dir / "processed" / "val-one-hot.csv")
test = pd.read_csv(data_dir / "processed" / "test-one-hot.csv")

In [ ]:
train_sds = StandardDataset(
    train,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["sex"],
    privileged_classes=[[1]],
)
test_sds = StandardDataset(
    test,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["sex"],
    privileged_classes=[[1]],
)
val_sds = StandardDataset(
    val,
    label_name="salary",
    favorable_classes=[1],
    protected_attribute_names=["sex"],
    privileged_classes=[[1]],
)
index = train_sds.feature_names.index("sex")

In [ ]:
privileged_groups = [{"sex": 1.0}]
unprivileged_groups = [{"sex": 0.0}]

## Load original model

In [ ]:
bl_model = joblib.load(artifacts_dir / "models" / "finance" / "baseline.pkl")

bl_val_pred = bl_model.predict(val.drop("salary", axis=1))

val_sds_pred = val_sds.copy(deepcopy=True)
val_sds_pred.labels = bl_val_pred.reshape(-1, 1)

## Perform intervention

In [ ]:
# Learn parameters to equalize odds and apply to create a new dataset
eopp = EqOddsPostprocessing(
    privileged_groups=privileged_groups,
    unprivileged_groups=unprivileged_groups,
    seed=np.random.seed(),
)
eopp = eopp.fit(val_sds, val_sds_pred)
val_sds_pred_tranf = eopp.predict(val_sds_pred)

In [ ]:
val_sds_pred_tranf.scores = val_sds_pred_tranf.labels

## Analyse accuracy and fairness

In [ ]:
fnr = np.abs(
    val_sds_pred_tranf.scores[(val.salary == 1) & (val.sex == 1)].mean()
    - val_sds_pred_tranf.scores[(val.salary == 1) & (val.sex == 0)].mean()
)
fpr = np.abs(
    val_sds_pred_tranf.scores[(val.salary == 0) & (val.sex == 1)].mean()
    - val_sds_pred_tranf.scores[(val.salary == 0) & (val.sex == 0)].mean()
)

In [ ]:
print("Accuracy =", accuracy(val_sds_pred_tranf.scores.flatten(), val.salary))
print(
    "Female accuracy =",
    accuracy(
        val_sds_pred_tranf.scores.flatten()[val.sex == 0],
        val.salary[val.sex == 0],
    ),
)
print(
    "Male accuracy =",
    accuracy(
        val_sds_pred_tranf.scores.flatten()[val.sex == 1],
        val.salary[val.sex == 1],
    ),
)
print("FNR =", fnr)
print("FPR =", fpr)

In [ ]:
print(
    "Equalised odds = ",
    equalised_odds_p(val_sds_pred_tranf.scores.flatten(), val.sex, val.salary),
)

### Plots

In [ ]:
eo_bar = go.Figure(
    data=[
        go.Bar(
            x=[label],
            y=[
                val_sds_pred_tranf.scores[
                    (val.sex == sex) & (val.salary == label)
                ].mean()
            ],
            name="Male" if sex else "Female",
        )
        for label in range(2)
        for sex in range(2)
    ]
)
eo_bar

In [ ]:
export_plot(eo_bar, "hardt-eo.json")